# Information Extraction
- 06/28/30 Extract the entity relations related to covid-19
    - Whether there is relationship or not
    - If so, what kind, and how likely
    - Use the openNRE to do so

- Examples: 
    - Risk Factors
        - “The two-way interaction between COVID-19 and diabetes mellitus sets up a vicious cycle wherein COVID-19 leads to worsening of dysglycemia and diabetes mellitus”
            1.	Covid-19 has relationship with diabetes. 
            2.	Covid-19 worsen diabetes
            3.	Does not indicate diabetes is a risk factor; but indicate covid-19 impact diabetes. 
    - Efficacy of therapeutics and interventions
        - i. “Increasing eosinophils may be an indicator of COVID-19 improvement. The COVID-19 patients may benefit from sustained lopinavir use.”
            1.	lopinavir may be efficient for treating covid-19.
            2.	OpenNRE: “eosinophils” has part (P527)  “covid-19”

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import opennre

# Example

In [11]:
model = opennre.get_model('wiki80_bert_softmax')

RuntimeError: unexpected EOF, expected 9393423 more bytes. The file might be corrupted.

In [10]:
opennre.get_model

<function opennre.pretrain.get_model(model_name, root_path='/home/ubuntu/.opennre')>

In [9]:
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'

model.infer({'text': , 
             'h': {'pos': (18, 46)}, 
             't': {'pos': (78, 91)}
            })




('father', 0.7500484585762024)